# 基于协同过滤的临床疾病辅助诊断方法研究

## 1.数据预处理

- 导入数据，数据清洗
- 数据变换，连续属性离散化
- 数据可视化

### 1.1 导入数据，数据清洗

In [1]:
# -*- coding:utf-8 -*-
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
def load_dermatology_dataset(filename):
    data=pd.read_csv(filename,delimiter=',')
    target_names_nums={'psoriasis':112,'seboreic dermatitis':61,'lichen planus':72,'pityriasis rosea':49,'cronic dermatitis':52,'pityriasis rubra pilaris':20}
    target_names=['psoriasis','seboreic dermatitis','lichen planus', 'pityriasis rosea','cronic dermatitis','pityriasis rubra pilaris']
    target_nums=[112,61,72,49,52,20]
    target_labels=[]
    if data.shape[1]==34:
        y_num=pd.read_csv('/home/min/bigdata/data/ex1/data/dermatologyNumric.csv',delimiter=',')['target-label-num']
        data=data.jion(y_num)
    y_num=data['target-label-num']
    for c in y_num:
        target_labels.append(target_names[int(c-1)])
    plot_data=data.copy()
    y_class=pd.DataFrame(target_labels,columns=['target-label'])
    plot_data['target-label']=y_class
    return plot_data,data

### 1.2 数据变换，连续属性离散化变成分类属性

In [37]:
def get_category(data,k=6):
    ages_with_nan=data[' Age (linear)']
    ages=sorted(ages_with_nan.dropna().values)
    rate_list=[]
    for i in range(1,k):
        rate_list.append(i/float(k))
    rate=np.array(rate_list)
    qs=len(ages)*rate
    qs1=np.ceil(qs)
    qs2=np.floor(qs)
    qs1=qs1.astype(np.int32)
    qs2=qs2.astype(np.int32)
    qss=[]
    for i in range(len(rate)):
        qss.append((ages[qs1[i]]+ages[qs2[i]])/2)
    def change2category(x):
        if not np.isnan(x):
            c=0
            while(c<len(qss) and x>qss[c]):
                c+=1
            return c
        return x
    ages_c=map(change2category,ages_with_nan.values)
    new_ages=pd.Series(ages_c)
    data[' Age (linear)']=new_ages
    return data

### 1.3 数据可视化

In [46]:
from pandas.tools.plotting import parallel_coordinates
def plot_data(plot_data):
    plt.figure(figsize=(15,8))
    parallel_coordinates(plot_data,'target-label')
    plt.show()

### 1.4 处理类别不平衡状态

In [48]:
from smote import Smote
def get_smote_sampledata(data):
    x=data.iloc[:,:-1].values
    y=data.iloc[:,-1].values #数字类别
    target_nums=[112,61,72,49,52,20]
    mx=target_nums[0]
    x_sample=x[y==1]
    y_sample=np.ones((len(x_sample),1))
    for i in range(1,6):
        srate=int(np.floor(mx/target_nums[i]))
        print srate
        st=Smote(sampling_rate=srate, k=6)
        xo=(st.fit(x[y==(i+1)]))
        x_sample=np.concatenate((x_sample,xo), axis=0)
        y_sample=np.concatenate((y_sample,(i+1)*np.ones((len(xo),1))), axis=0)
    data_sample=np.column_stack((x_sample,y_sample))
    return x_sample,y_sample

## 2. 实验

In [ ]:
# 从sklearn.cross_validation中导入train_test_split用于数据分割。
from sklearn.cross_validation import train_test_split

# 从sklearn.preprocessing里导入数据标准化模块。
from sklearn.preprocessing import StandardScaler

# 从sklearn.metrics里导入classification_report模块。
from sklearn.metrics import classification_report

# 从sklearn.linear_model里导入LogisticRegression与SGDClassifier。
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

# 从sklearn.svm里导入基于线性假设的支持向量机分类器LinearSVC。
from sklearn.svm import LinearSVC

#多层感知器MLP
from sklearn.neural_network import MLPClassifier

# 对类别型特征进行转化，成为特征向量。
from sklearn.feature_extraction import DictVectorizer

target_names_nums={'psoriasis':112,'seboreic dermatitis':61,'lichen planus':72,'pityriasis rosea':49,'cronic dermatitis':52,'pityriasis rubra pilaris':20}
target_names=['psoriasis','seboreic dermatitis','lichen planus', 'pityriasis rosea','cronic dermatitis','pityriasis rubra pilaris']
target_nums=[112,61,72,49,52,20]

#读入数据
x_df=pd.DataFrame(x_sample,columns=feature_names)
y_df=pd.DataFrame(y_sample,columns=['target-label'])



# 随机采样20%的数据用于测试，剩下的80%用于构建训练集合。
X_train, X_test, y_train, y_test = train_test_split(x_df,y_df, test_size=0.2, random_state=33)
print pd.value_counts(y_train.values.flatten())
print pd.value_counts(y_test.values.flatten())


# 初始化LogisticRegression与SGDClassifier。
lr = LogisticRegression()
sgdc = SGDClassifier()

# 调用LogisticRegression中的fit函数/模块用来训练模型参数。
lr.fit(X_train, y_train)
# 使用训练好的模型lr对X_test进行预测，结果储存在变量lr_y_predict中。
lr_y_predict = lr.predict(X_test)

# 使用逻辑斯蒂回归模型自带的评分函数score获得模型在测试集上的准确性结果。
print 'Accuracy of LR Classifier:', lr.score(X_test, y_test)
# 利用classification_report模块获得LogisticRegression其他三个指标的结果。
print classification_report(y_test, lr_y_predict, target_names=target_names)




# 初始化线性假设的支持向量机分类器LinearSVC。
lsvc = LinearSVC()
#进行模型训练
lsvc.fit(X_train, y_train)
# 利用训练好的模型对测试样本的数字类别进行预测，预测结果储存在变量y_predict中。
y_predict = lsvc.predict(X_test)
# 使用模型自带的评估函数进行准确性测评。
print 'The Accuracy of SVM is', lsvc.score(X_test, y_test)
print classification_report(y_test, y_predict, target_names=target_names)



# 初始化多层感知器MLP
mlp = MLPClassifier(solver='lbfgs', alpha=1e-5,  
                    hidden_layer_sizes=(6, 6), random_state=1)
mlp.fit(X_train, y_train)
mlp_y_pred=mlp.predict(X_test) 

# 输出多层感知分类器在测试集上的分类准确性，以及更加详细的精确率、召回率、F1指标。
print 'The accuracy of MLP Classifier is', mlp.score(X_test, y_test)
print classification_report(mlp_y_pred, y_test,target_names=target_names)



# 使用随机森林分类器RF
# 对类别型特征进行转化，成为特征向量。
vec = DictVectorizer(sparse=False)
X_train = vec.fit_transform(X_train.to_dict(orient='record'))
X_test = vec.transform(X_test.to_dict(orient='record'))

# 使用随机森林分类器进行集成模型的训练以及预测分析。
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_y_pred = rfc.predict(X_test)

# 输出随机森林分类器在测试集上的分类准确性，以及更加详细的精确率、召回率、F1指标。
print 'The accuracy of random forest classifier is', rfc.score(X_test, y_test)
print classification_report(rfc_y_pred, y_test,target_names=target_names)
